In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from matplotlib import rc
import torch
import random
import matplotlib.animation as animation

In [2]:
def nth_derivative(f, wrt, n): 
# adapted from the code of Zhuo Shen
# calculate nth derivative of f against wrt
  for i in range(n):
    grads = torch.autograd.grad(f, wrt, create_graph=True, allow_unused=True)[0]
    f = grads.sum()
    if grads is None:
      print('bad grad')
      return torch.tensor(0.)
  return grads

In [3]:
# parameters
Vlid = 1.0
Lx = 2.0
Ly = 1.0
visc = 1.0
rho = 1.0
nu = visc / rho
relax = 0.5
vort_init = 0.0
N = 50
M = 2000
# BC for psi=0
psi_x_samples = []
psi_y_samples = []
psi_samples = []
for n in range(N):
  psi_x_samples.append(0.0)
  psi_y_samples.append(random.uniform(0.0,Ly))
  psi_samples.append(0.0)
for n in range(N):
  psi_x_samples.append(Lx)
  psi_y_samples.append(random.uniform(0.0,Ly))
  psi_samples.append(0.0)
for n in range(N):
  psi_x_samples.append(random.uniform(0.0,Lx))
  psi_y_samples.append(0.0)
  psi_samples.append(0.0)
for n in range(N):
  psi_x_samples.append(random.uniform(0.0,Lx))
  psi_y_samples.append(Ly)
  psi_samples.append(0.0)
psi_x_samples = np.array(psi_x_samples).T
psi_y_samples = np.array(psi_y_samples).T
psi_samples = np.array(psi_samples).T
# BC for dpsi/dy
dypsi_x_samples = []
dypsi_y_samples = []
dypsi_samples = []
for n in range(N):
  dypsi_x_samples.append(random.uniform(0.0,Lx))
  dypsi_y_samples.append(0.0)
  dypsi_samples.append(0.0)
for n in range(N):
  dypsi_x_samples.append(random.uniform(0.0,Lx))
  dypsi_y_samples.append(Ly)
  dypsi_samples.append(Vlid)
dypsi_x_samples = np.array(dypsi_x_samples).T
dypsi_y_samples = np.array(dypsi_y_samples).T
dypsi_samples = np.array(dypsi_samples).T
# BC for dpsi/dx=0
dxpsi_x_samples = []
dxpsi_y_samples = []
dxpsi_samples = []
for n in range(N):
  dxpsi_x_samples.append(0.0)
  dxpsi_y_samples.append(random.uniform(0.0,Ly))
  dxpsi_samples.append(0.0)
for n in range(N):
  dxpsi_x_samples.append(Lx)
  dxpsi_y_samples.append(random.uniform(0.0,Ly))
  dxpsi_samples.append(0.0)
dxpsi_x_samples = np.array(dxpsi_x_samples).T
dxpsi_y_samples = np.array(dxpsi_y_samples).T
dxpsi_samples = np.array(dxpsi_samples).T
# BC for omega at the sides
omegax_x_samples = []
omegax_y_samples = []
omegax_samples = []
for n in range(N):
  omegax_x_samples.append(0.0)
  omegax_y_samples.append(random.uniform(0.0,Ly))
  omegax_samples.append(0.0) # omega plus the derivative equals zero
for n in range(N):
  omegax_x_samples.append(Lx)
  omegax_y_samples.append(random.uniform(0.0,Ly))
  omegax_samples.append(0.0)
omegax_x_samples = np.array(omegax_x_samples).T
omegax_y_samples = np.array(omegax_y_samples).T
omegax_samples = np.array(omegax_samples).T
# BC for omega at the top and bottom
omegay_x_samples = []
omegay_y_samples = []
omegay_samples = []
for n in range(N):
  omegay_x_samples.append(random.uniform(0.0,Lx))
  omegay_y_samples.append(0.0)
  omegay_samples.append(0.0)
for n in range(N):
  omegay_x_samples.append(random.uniform(0.0,Lx))
  omegay_y_samples.append(Ly)
  omegay_samples.append(0.0)
omegay_x_samples = np.array(omegay_x_samples).T
omegay_y_samples = np.array(omegay_y_samples).T
omegay_samples = np.array(omegay_samples).T
# random collocation points
coll_x_samples = []
coll_y_samples = []
coll_samples = []
for m in range(M):
  coll_x_samples.append(random.uniform(0.0,Lx))
  coll_y_samples.append(random.uniform(0.0,Ly))
  coll_samples.append(0.0)
coll_x_samples = np.array(coll_x_samples).T
coll_y_samples = np.array(coll_y_samples).T
coll_samples = np.array(coll_samples).T

In [4]:
# pack everything into tensors
psi_x = torch.tensor(psi_x_samples).float()
psi_x.requires_grad = True
psi_y = torch.tensor(psi_y_samples).float()
psi_y.requires_grad = True
psi = torch.tensor(psi_samples).float()
psi.requires_grad = True
dypsi_x = torch.tensor(dypsi_x_samples).float()
dypsi_x.requires_grad = True
dypsi_y = torch.tensor(dypsi_y_samples).float()
dypsi_y.requires_grad = True
dypsi = torch.tensor(dypsi_samples).float()
dypsi.requires_grad = True
dxpsi_x = torch.tensor(dxpsi_x_samples).float()
dxpsi_x.requires_grad = True
dxpsi_y = torch.tensor(dxpsi_y_samples).float()
dxpsi_y.requires_grad = True
dxpsi = torch.tensor(dxpsi_samples).float()
dxpsi.requires_grad = True
omegax_x = torch.tensor(omegax_x_samples).float()
omegax_x.requires_grad = True
omegax_y = torch.tensor(omegax_y_samples).float()
omegax_y.requires_grad = True
omegax = torch.tensor(omegax_samples).float()
omegax.requires_grad = True
omegay_x = torch.tensor(omegay_x_samples).float()
omegay_x.requires_grad = True
omegay_y = torch.tensor(omegay_y_samples).float()
omegay_y.requires_grad = True
omegay = torch.tensor(omegay_samples).float()
omegay.requires_grad = True
coll_x = torch.tensor(coll_x_samples).float()
coll_x.requires_grad = True
coll_y = torch.tensor(coll_y_samples).float()
coll_y.requires_grad = True
coll = torch.tensor(coll_samples).float()
coll.requires_grad = True

In [5]:
class PINN(torch.nn.Module): 
  def __init__(self):
    super(PINN,self).__init__()
    self.m1 = torch.nn.Linear(2,20)
    self.m2 = torch.nn.Linear(20,200)
    self.m3 = torch.nn.Linear(200,200)
    self.m4 = torch.nn.Linear(200,200)
    self.m5 = torch.nn.Linear(200,200)
    self.m6 = torch.nn.Linear(200,200)
    self.m7 = torch.nn.Linear(200,20)
    self.m8 = torch.nn.Linear(20,2)
    # change the number of layers or neurons if needed
  def forward(self,x,y):
    input = torch.cat((x.view(x.size(0), -1), y.view(y.size(0), -1)), dim=1)
    layer_1 = torch.tanh(self.m1(input))
    layer_2 = torch.tanh(self.m2(layer_1))
    layer_3 = torch.tanh(self.m3(layer_2))
    layer_4 = torch.tanh(self.m4(layer_3))
    layer_5 = torch.tanh(self.m5(layer_4))
    layer_6 = torch.tanh(self.m6(layer_5))
    layer_7 = torch.tanh(self.m6(layer_6))
    layer_8 = torch.tanh(self.m6(layer_7))
    # change the activation function if needed
    return layer_8

In [6]:
def eqn1(omega,psi,x,y): # PDE #1
  psi_xx = nth_derivative(psi.sum(), x, 2)
  psi_yy = nth_derivative(psi.sum(), y, 2)
  return omega + psi_xx + psi_yy
def eqn2(omega,psi,x,y):
  omega_x = nth_derivative(omega.sum(), x, 1)
  omega_xx = nth_derivative(omega.sum(), x, 2)
  omega_y = nth_derivative(omega.sum(), y, 1)
  omega_yy = nth_derivative(omega.sum(), y, 2)
  psi_x = nth_derivative(psi.sum(), x, 1)
  psi_y = nth_derivative(psi.sum(), y, 1)
  r = omega_xx + omega_yy - 1/nu*(torch.mul(psi_y, omega_x) - torch.mul(psi_x, omega_y))
  return r
def omega_bdry_x(omega,psi,x): # omega BC at the sides
  return omega + nth_derivative(psi.sum(), x, 2)
def omega_bdry_y(omega,psi,y): # omega BC at the top and bottom
  return omega + nth_derivative(psi.sum(), y, 2)
def find_psi_x(psi,x):
  return nth_derivative(psi.sum(), x, 1)
def find_psi_y(psi,y):
  return nth_derivative(psi.sum(), y, 1)
def find_psi_xx(psi,x):
  return nth_derivative(psi.sum(), x, 2)
def find_psi_yy(psi,y):
  return nth_derivative(psi.sum(), y, 2)

In [7]:
def train(model, optimizer, criterion, epochs):
  # first output is omega, second is psi
  for e in range(1,epochs+1):
    model.train()
    q = model(psi_x,psi_y)
    psi_out = q[:,1]
    psi_out = torch.reshape(psi_out,(-1,))
    psi_loss = criterion(psi_out, psi)
    r = model(dypsi_x,dypsi_y)
    psi_out = r[:,1]
    psi_out_y = find_psi_y(psi_out,dypsi_y)
    dypsi_loss = criterion(psi_out_y,dypsi)
    s = model(dxpsi_x,dxpsi_y)
    psi_out = s[:,1]
    psi_out_x = find_psi_x(psi_out,dxpsi_x)
    dxpsi_loss = criterion(psi_out_x,dxpsi)
    t = model(omegax_x,omegax_y)
    omega_out = t[:,0]
    psi_out = t[:,1]
    omega_res = omega_bdry_x(omega_out,psi_out,omegax_x)
    omegax_loss = criterion(omega_res, omegax)
    u = model(omegay_x,omegay_y)
    omega_out = u[:,0]
    psi_out = u[:,1]
    omega_res = omega_bdry_y(omega_out,psi_out,omegay_y)
    omegay_loss = criterion(omega_res, omegay)
    v = model(coll_x,coll_y)
    omega_out = v[:,0]
    psi_out = v[:,1]
    res1 = eqn1(omega_out,psi_out,coll_x,coll_y)
    eqn1_loss = criterion(res1, coll)
    res2 = eqn2(omega_out,psi_out,coll_x,coll_y)
    eqn2_loss = criterion(res2, coll)
    total_loss = psi_loss + dypsi_loss + dxpsi_loss + omegax_loss + omegay_loss + eqn1_loss + eqn2_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    print(f"| epoch {e:2d} | loss {total_loss:.6f}")

In [8]:
net = PINN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.001)
criterion = torch.nn.MSELoss()
train(net,optimizer,criterion,500)

| epoch  1 | loss 0.504445
| epoch  2 | loss 0.415252
| epoch  3 | loss 0.382897
| epoch  4 | loss 0.343333
| epoch  5 | loss 0.321757
| epoch  6 | loss 0.300375
| epoch  7 | loss 0.298939
| epoch  8 | loss 0.291968
| epoch  9 | loss 0.280804
| epoch 10 | loss 0.269172
| epoch 11 | loss 0.257274
| epoch 12 | loss 0.253905
| epoch 13 | loss 0.248873
| epoch 14 | loss 0.240218
| epoch 15 | loss 0.232889
| epoch 16 | loss 0.226341
| epoch 17 | loss 0.220393
| epoch 18 | loss 0.216826
| epoch 19 | loss 0.210064
| epoch 20 | loss 0.208189
| epoch 21 | loss 0.210017
| epoch 22 | loss 0.218873
| epoch 23 | loss 0.205939
| epoch 24 | loss 0.196054
| epoch 25 | loss 0.199502
| epoch 26 | loss 0.205773
| epoch 27 | loss 0.207387
| epoch 28 | loss 0.193442
| epoch 29 | loss 0.199342
| epoch 30 | loss 0.203157
| epoch 31 | loss 0.192036
| epoch 32 | loss 0.198060
| epoch 33 | loss 0.195875
| epoch 34 | loss 0.191753
| epoch 35 | loss 0.197812
| epoch 36 | loss 0.189365
| epoch 37 | loss 0.192155
|

In [9]:
# plot the solution (adapted from Zheng Tan's code)
M = 1000
dx = Lx / M
dy = Ly / M
x = np.linspace(0, Lx, M+1)
y = np.linspace(0, Ly, M+1)
X, Y = np.meshgrid(np.linspace(0, Lx, M+1), np.linspace(0, Ly, M+1))
long_x = np.zeros((M+1)*(M+1))
for i in range((M+1)*(M+1)):
  a = i // (M+1) 
  long_x[i] = a*dx
long_y = np.zeros((M+1)*(M+1))
for j in range((M+1)*(M+1)):
  b = j % (M+1) 
  long_y[j] = b*dy
long_x_tensor = torch.from_numpy(long_x).float()
long_x_tensor.requires_grad = True
long_y_tensor = torch.from_numpy(long_y).float()
long_y_tensor.requires_grad = True
q = net(long_x_tensor, long_y_tensor)
psi_tensor = q[:,1]
long_psi = psi_tensor.detach().numpy()
psi = np.zeros((M+1,M+1))
for i in range(len(long_psi)):
  a = i // (M+1)
  b = i % (M+1)
  psi[a,b] = long_psi[i]
fig, ax = plt.subplots()
ax.cla()
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
ax.set_xlabel('x')
ax.set_ylabel('y')
fig = plt.contour(x, y, psi, 100)
plt.savefig('psi contour.png')
from google.colab import files
files.download('psi contour.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>